# Neuronexus Analysis v6

**PLX → LFP Filtering → Spike Sorting → Phase Locking**

このノートブックでは、PLXファイルを読み込んでLFP解析・スパイクソーティング・位相ロック解析を順番に実行します。  
各セクションのON/OFFは下の設定セルで切り替えてください。

## 0. 設定

In [ ]:
import os
os.add_dll_directory(r"c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\torch\lib")
import torch
print("PyTorch:", torch.__version__, "CUDA:", torch.cuda.is_available())
import sys
import numpy as np

# ============================================================
# 解析設定 - 必要に応じて変更してください
# ============================================================

# PLXファイルパス（空文字列 → GUIで選択）
PLX_FILE = r"\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\260114-4_Wildtype_S1BF_920_wisker\260114-4_Wildtype_S1BF_920_wisker.plx"

# 出力ディレクトリ（空文字列 → PLXファイルと同じフォルダ）
OUTPUT_DIR = r"\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\0209解析"

# 各ステップのON/OFF
RUN_LFP    = True    # LFP処理を実行するか
RUN_SPIKE  = True    # スパイクソーティングを実行するか
RUN_PHASE  = True    # 位相ロック解析を実行するか

# Widebandデータを読み込むか（メモリ節約のためFalseにできる）
LOAD_WIDEBAND = True

## 1. PLXファイル読み込み

In [ ]:


# NumPy 2.0+ 互換パッチ（neo/quantities より先に実行）
import numpy_compat

# Matplotlib をノートブック用に設定
# GUIウィンドウを使うので %matplotlib tk を使用
%matplotlib tk

print("=" * 60)
print("  Neuronexus Analysis v6 (Notebook)")
print("=" * 60)

  Neuronexus Analysis v6 (Notebook)


In [6]:
# PLXファイル選択
plx_file = PLX_FILE
if not plx_file:
    from get_path import get_path
    plx_file = get_path(mode='file', file_type='plx', initial_dir=r'\\Nagoya_AMCBNAS2\wakelab7\okita\whisker_analysis\20260114-1_Wildtype_S1BF\260114-4_Wildtype_S1BF_920_wisker')

if not plx_file or not os.path.exists(plx_file):
    raise FileNotFoundError(f"PLXファイルが見つかりません: {plx_file}")

print(f"\n[Step 0] PLX読み込み: {os.path.basename(plx_file)}")
from data_loader import load_plx
plx_data = load_plx(plx_file, load_wideband=LOAD_WIDEBAND)

if OUTPUT_DIR:
    plx_data.output_dir = OUTPUT_DIR
output_dir = plx_data.output_dir
basename = plx_data.basename

# 結果を保持
lfp_results = None
spike_results = None
phase_results = None

print(f"\n出力先: {output_dir}")

 選択: 260114-4_Wildtype_S1BF_920_wisker.plx

[Step 0] PLX読み込み: 260114-4_Wildtype_S1BF_920_wisker.plx
[DataLoader] 読み込み: 260114-4_Wildtype_S1BF_920_wisker.plx


Parsing signal channels: 100%|██████████| 128/128 [00:00<00:00, 12083.25it/s]
Parsing spike channels: 16it [00:00, ?it/s]
Parsing event channels: 100%|██████████| 43/43 [00:00<?, ?it/s]


  LFP: (112445, 2), fs=1000Hz
  LFP: (112460, 16), fs=1000Hz
  Wideband: (4498350, 16), fs=40000Hz
  フレーム同期: EVT03, 1241個
  刺激イベント: 90個
  動画: 1241 フレーム
  Trim: 7.71s ~ 106.89s
  スパイク: 16 trains (0 sorted)
[DataLoader] 完了: 112.5秒のデータ

出力先: //Nagoya_AMCBNAS2/wakelab7/okita/whisker_analysis/20260114-1_Wildtype_S1BF/260114-4_Wildtype_S1BF_920_wisker


## 2. LFP Filtering

LFP Filter GUI が起動します。`plx_data` のファイルパス・チャンネル情報等は自動でGUIに反映されます。  
フィルタ設定を行い「実行」ボタンを押してください。GUI を閉じると次のセルに進めます。

In [7]:
if RUN_LFP:
    print("[Step 1] LFP Filter GUI を起動...")

    from lfp_filter_gui import LfpFilterGUI

    lfp_config_holder = [None]

    def on_lfp_done(config, _plx_data):
        lfp_config_holder[0] = config

    # plx_data を渡すと、ファイルパス・データ情報が自動でGUIに反映される
    gui = LfpFilterGUI(plx_data=plx_data, on_done=on_lfp_done)
    gui.run()

    lfp_config = lfp_config_holder[0]
    if lfp_config is None:
        print("  ⚠ LFP処理がキャンセルされました")
    else:
        print("[Step 1b] LFPパイプライン実行...")
        from lfp_pipeline import run_lfp_pipeline
        lfp_results = run_lfp_pipeline(lfp_config, plx_data)
        print("  ✓ LFP処理完了")
else:
    print("[Step 1] LFP処理: スキップ")

[Step 1] LFP Filter GUI を起動...
[LFP GUI] 設定読み込み: \\nagoya_amcbnas2\wakelab7\okita\GitHub\whisker_analysis\neuronexus_analysis_v6\lfp_config.json
[Step 1b] LFPパイプライン実行...
=== LFPパイプライン開始: 260114-4_Wildtype_S1BF_920_wisker ===
  LFP: (112460, 16), fs=1000Hz
  Trim: 7.7~106.9s

[1/6] フィルタリング...
  バンドパス: 0.1-100.0Hz [FIR]

  高調波ノイズ除去...
  高調波ノイズ除去: [10.0, 30.0, 50.0, 70.0] Hz
  除去: ['10Hz', '30Hz', '50Hz', '70Hz']

[2/6] チャンネル処理...
    異常チャンネル: D7(Ch4), var=0.033510
  除外: [7], 残り: 15ch

[3/6] モーション解析...
  ROIを選択 (Enter確定, C取消)
  ノイズ区間: 28.8%

[4/6] ICA...
    IC9: ratio=16.11 → 除去
    IC1: ratio=12.57 → 除去
    IC4: ratio=9.18 → 除去
    IC8: ratio=4.49 → 除去
  除去: 4 成分

[5/6] パワー解析...
  刺激イベント: 90個

[6/6] プロット出力...

  CSV保存...

=== LFPパイプライン完了 ===
  ✓ LFP処理完了


## 3. Spike Sorting

Spike Sorting GUI が起動します。各チャンネルのスパイクソーティングを行ってください。

In [ ]:
if RUN_SPIKE and plx_data.wideband_raw is not None:
    print("[Step 2] Spike Sorting GUI を起動...")

    from spike_sort_gui import SpikeSortGUI

    spike_holder = [None]

    def on_spike_done(results):
        spike_holder[0] = results

    gui = SpikeSortGUI(
        wideband_data=plx_data.wideband_raw,
        fs=plx_data.wideband_fs,
        output_dir=output_dir,
        basename=basename,
        on_done=on_spike_done
    )
    gui.run()
    spike_results = spike_holder[0]

    if spike_results:
        from spike_plotting import plot_spike_grand_summary, plot_quality_table
        plot_spike_grand_summary(spike_results, output_dir, basename,
                                 show=True, save=True)
        plot_quality_table(spike_results, output_dir, basename,
                           show=True, save=True)
        n_units = sum(len(r.units) for r in spike_results.values())
        print(f"  ✓ スパイクソーティング完了: {n_units} units")
    else:
        print("  ⚠ スパイクソーティングがキャンセルされました")

elif not RUN_SPIKE:
    print("[Step 2] スパイクソーティング: スキップ")
else:
    print("[Step 2] Widebandデータなし - スパイクソーティングをスキップ")

[Step 2] Spike Sorting GUI を起動...


## 4. Phase Locking Analysis

位相ロック解析GUI が起動します。解析帯域や条件を設定してください。

In [ ]:
if RUN_PHASE and lfp_results is not None:
    print("[Step 3] Phase Locking GUI を起動...")

    condition_masks = None
    if all(k in lfp_results for k in ['clean_baseline', 'clean_stim', 'clean_post']):
        condition_masks = {
            'baseline': lfp_results['clean_baseline'],
            'stim': lfp_results['clean_stim'],
            'post': lfp_results['clean_post']
        }

    from phase_gui import PhaseGUI

    phase_holder = [None, None]

    def on_phase_done(pr, cr):
        phase_holder[0] = pr
        phase_holder[1] = cr

    gui = PhaseGUI(
        lfp_cleaned=lfp_results.get('lfp_cleaned'),
        lfp_times=lfp_results.get('lfp_times'),
        fs=lfp_results.get('fs'),
        segment=plx_data.segment,
        spike_results=spike_results,
        stim_times=lfp_results.get('stim_times'),
        condition_masks=condition_masks,
        original_ch_numbers=lfp_results.get('original_ch_numbers', []),
        output_dir=output_dir,
        basename=basename,
        on_done=on_phase_done
    )
    gui.run()
    phase_results = phase_holder[0]

    if phase_results:
        print("  ✓ 位相ロック解析完了")
    else:
        print("  ⚠ 位相ロック解析がキャンセルされました")

elif not RUN_PHASE:
    print("[Step 3] 位相ロック解析: スキップ")
else:
    print("[Step 3] LFPデータなし - 位相ロック解析をスキップ")

[Step 3] Phase Locking GUI を起動...
  ユニット: ch0_unit1 (141 spikes)
  theta (4.0-12.0 Hz)...
    Ch0: MRL=0.108, PPC=0.004, p=0.2035, n=136
    Ch1: MRL=0.080, PPC=-0.001, p=0.4192, n=136
    Ch2: MRL=0.149, PPC=0.015, p=0.0486*, n=136
    Ch3: MRL=0.108, PPC=0.004, p=0.2025, n=136
    Ch4: MRL=0.122, PPC=0.008, p=0.1304, n=136
    Ch5: MRL=0.129, PPC=0.009, p=0.1044, n=136
    Ch6: MRL=0.115, PPC=0.006, p=0.1649, n=136
    Ch7: MRL=0.106, PPC=0.004, p=0.2167, n=136
    Ch8: MRL=0.121, PPC=0.007, p=0.1350, n=136
    Ch9: MRL=0.077, PPC=-0.001, p=0.4466, n=136
    Ch10: MRL=0.097, PPC=0.002, p=0.2767, n=136
    Ch11: MRL=0.091, PPC=0.001, p=0.3234, n=136
    Ch12: MRL=0.093, PPC=0.001, p=0.3073, n=136
    Ch13: MRL=0.094, PPC=0.001, p=0.3012, n=136
    Ch14: MRL=0.092, PPC=0.001, p=0.3125, n=136
  gamma (30.0-80.0 Hz)...
    Ch0: MRL=0.101, PPC=0.003, p=0.2516, n=136
    Ch1: MRL=0.136, PPC=0.011, p=0.0808, n=136
    Ch2: MRL=0.101, PPC=0.003, p=0.2518, n=136
    Ch3: MRL=0.132, PPC=0.010,

KeyboardInterrupt: 

## 5. 完了

In [ ]:
print("=" * 60)
print("  Neuronexus Analysis v6 完了!")
print(f"  出力先: {output_dir}")
print("=" * 60)

print("\n--- 結果サマリー ---")
if lfp_results is not None:
    n_ch = lfp_results.get('lfp_cleaned', np.empty((0,0))).shape[1] if lfp_results.get('lfp_cleaned') is not None else 0
    print(f"  LFP: {n_ch} チャンネル処理済み")
else:
    print("  LFP: 未処理")

if spike_results is not None:
    n_units = sum(len(r.units) for r in spike_results.values())
    print(f"  スパイク: {n_units} units ({len(spike_results)} チャンネル)")
else:
    print("  スパイク: 未処理")

if phase_results is not None:
    print(f"  位相ロック: 解析完了")
else:
    print("  位相ロック: 未処理")2

  Neuronexus Analysis v6 完了!
  出力先: //Nagoya_AMCBNAS2/wakelab7/okita/whisker_analysis/20260114-1_Wildtype_S1BF/260114-4_Wildtype_S1BF_920_wisker

--- 結果サマリー ---
  LFP: 15 チャンネル処理済み
  スパイク: 44 units (16 チャンネル)
  位相ロック: 解析完了


In [ ]:
  import sys
  print("Python:", sys.executable)

  # KiloSort直接テスト
  try:
      import kilosort
      print("KiloSort:", kilosort.__version__)
  except Exception as e:
      print("KiloSort import ERROR:", e)

  # PyTorch/CUDAテスト
  try:
      import torch
      print("PyTorch:", torch.__version__)
      print("CUDA:", torch.cuda.is_available())
  except Exception as e:
      print("PyTorch import ERROR:", e)

  # wrapper テスト
  try:
      from kilosort_wrapper import is_kilosort_available
      print("is_kilosort_available():", is_kilosort_available())
  except Exception as e:
      print("wrapper ERROR:", e)

Python: c:\Users\wakel\anaconda3\envs\neoeeg\python.exe
KiloSort import ERROR: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.
PyTorch import ERROR: [WinError 127] The specified procedure could not be found. Error loading "c:\Users\wakel\anaconda3\envs\neoeeg\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.
is_kilosort_available(): False
